In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df =pd.read_csv("https://bit.ly/seoul-120-text-csv")
df.shape

(2645, 5)

In [3]:
df['문서'] = df['제목'] + df['내용']
df.head(3)

,번호,분류,제목,내용,내용번호,문서
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464,아빠 육아휴직 장려금아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려...
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045,[서울산업진흥원] 서울메이드란?서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하...
2,2643,환경,(강북구) 정비중,강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높이고 폐기물을 감...,23032485,(강북구) 정비중강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높...


In [4]:
df = df.query("분류.isin(['행정','경제','복지'])")

In [5]:
label_name = '분류'

In [6]:
feature = df['문서']
target = df[label_name]

In [7]:
# target_onehot = pd.get_dummies(target)
# from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [8]:
categories_array = np.array(target).reshape(-1, 1)
target_onehot = encoder.fit_transform(categories_array)

C:\Users\서울디지털인재개발원\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
target_onehot


array([1, 0, 1, ..., 2, 2, 1])

In [10]:
from sklearn.model_selection import train_test_split

feature_train, feature_test, target_train, target_test = train_test_split(feature,target_onehot, test_size=0.2, random_state=42, stratify=target_onehot)
feature_train.shape, feature_test.shape, target_train.shape, target_test.shape

((1710,), (428,), (1710,), (428,))

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 10000
oov_tok ='<oov>'
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer

In [12]:
tokenizer.fit_on_texts(feature_train)

In [13]:
word_to_index = tokenizer.word_index
sorted(word_to_index)[:10]

["'",
 "'25시",
 "'관리주체는",
 "'권장용도'란",
 "'노원'",
 "'당해",
 "'불허용도'란",
 "'새랑'의",
 "'성공창업",
 "'최대개발규모'라"]

In [14]:
# train_sequences = tokenizer.texts_to_sequences(feature_train)
# test_sequences = tokenizer.texts_to_sequences(feature_test)
train_sequences = tokenizer.texts_to_sequences([str(text) for text in feature_train])
test_sequences = tokenizer.texts_to_sequences([str(text) for text in feature_test])

In [15]:
max_length = 500
padding_type = 'post' # 뒤에 채우기

In [16]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
import pandas as pd
from run_model import compile_model

In [17]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [28]:
# 손실 함수를 초기화합니다.
loss_fn = nn.BCEWithLogitsLoss()

In [19]:
from keras.preprocessing.sequence import pad_sequences
feature_train_sp = pad_sequences(train_sequences,padding=padding_type,maxlen=max_length)
feature_test_sp = pad_sequences(test_sequences,padding=padding_type,maxlen=max_length)
print(feature_train.shape)

(1710,)


In [20]:
class_num = len(target.unique())
data_len = len(feature_train_sp[0])
input_data = feature_train_sp # 여기가 데이터 들어가는 곳

In [22]:
from LSTM_model import NeuralNetwork
model = NeuralNetwork(data_len,class_num)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [23]:
feature_train_sp = torch.tensor(feature_train_sp)
target_train = torch.tensor(target_train)

In [24]:
target_train

tensor([2, 0, 0,  ..., 2, 2, 0], dtype=torch.int32)

In [25]:
feature_train_sp

tensor([[ 900, 4196,  827,  ...,    0,    0,    0],
        [ 327, 6234,   79,  ...,    0,    0,    0],
        [2414, 6236, 2415,  ...,    0,    0,    0],
        ...,
        [   1,    1,    1,  ...,    0,    0,    0],
        [   1,    1,    1,  ...,    0,    0,    0],
        [1230, 4195,    1,  ...,    0,    0,    0]], dtype=torch.int32)

In [26]:
feature_train_sp.shape,target_train.shape

(torch.Size([1710, 500]), torch.Size([1710]))

In [30]:
target_train

tensor([2, 0, 0,  ..., 2, 2, 0], dtype=torch.int32)

In [38]:
import torch

# 클래스의 수 구하기
num_classes = len(torch.unique(target_train))

# 원-핫 인코딩 수행
target_train_onehot = torch.eye(num_classes)[target_train]

# 인코딩된 클래스의 수를 출력 차원으로 사용
output_dim = num_classes


In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# 임베딩 레이어를 제외한 나머지 레이어를 정의하는 클래스
class BiLSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(BiLSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.batchnorm = nn.BatchNorm1d(hidden_dim * 2)
        self.lstm2 = nn.LSTM(hidden_dim * 2, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, 16)
        self.fc2 = nn.Linear(16, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out1, _ = self.lstm1(embedded)
        lstm_out1 = self.batchnorm(lstm_out1)
        lstm_out2, _ = self.lstm2(lstm_out1)
        out = torch.relu(self.fc1(lstm_out2[:, -1, :]))
        out = self.fc2(out)
        return self.softmax(out)

# 데이터셋 클래스 정의
class TextDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

# 모델 및 데이터셋 인스턴스 생성
vocab_size = 10000  # 이 값은 Tokenizer에서 지정한 값과 같아야 합니다.
embedding_dim = 64
hidden_dim = 64
output_dim = target_train.shape[0]  # 클래스의 수

model = BiLSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# 데이터 로더 생성
class TrainDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], torch.argmax(self.targets[idx])

train_dataset = TrainDataset(feature_train_sp, target_train)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 모델 훈련
num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss}")


RuntimeError: running_mean should contain 500 elements not 128

In [33]:
print(target_train.shape)


torch.Size([1710])


In [ ]:

# 모델 예측
with torch.no_grad():
    outputs = model(feature_test_sp)
    _, predicted = torch.max(outputs, 1)
    print(predicted)
